In [1]:
import pymc3 as pm
import matplotlib.pyplot as plt
import numpy as np
import pickle
from pathlib import Path
import pandas as pd
import seaborn as sns

%matplotlib inline

In [2]:
from tqdm import tqdm
from collections import defaultdict as ddict
from pprint import pprint

In [14]:
def sortf(f):
    num = int(f.name.split("_")[-1].split(".")[0])
    return  num

def load_pkl(f):
    with open(f, "rb") as fi:
        data = pickle.load(fi)
    return data

In [4]:
def is_true(theta0, bs):
    low, up = bs
    return (theta0 > low) and (theta0 < up)


def rejrate(theta0, data, theta):
    reslist = [is_true(theta0, bs=[np.quantile(dat, q=theta), np.quantile(dat, q=1-theta)]) for dat in data] 
    return 1 - np.mean(reslist)


def getRatio(theta0, data):
    p1 = np.mean(data<=theta0)
    p2 = np.mean(data>theta0)
    res = np.min([p1, p2])
    return res



def getQuantile(theta0, data, alp=0.05):
    res = [getRatio(theta0, data=dat) for dat in data]
    return np.quantile(res, q=alp)

In [21]:
root = Path("./")
files = list(root.glob("Jeffrey_*"))
files = sorted(files, key=sortf)

In [22]:
Allres = ddict(ddict)
theta0 = 0

In [29]:
for fil in files:
    data = load_pkl(fil)
    num = sortf(fil)
    JEFdata = [dat["jef"]["theta"] for dat in data]
    Fulldata = [dat["full"]["theta"] for dat in data]
    
    Allres[num]["JEF"] = rejrate(theta0, JEFdata, theta=0.025)
    Allres[num]["FULL"] = rejrate(theta0, Fulldata, theta=0.025)


In [30]:
pprint(Allres)

defaultdict(<class 'collections.defaultdict'>,
            {0: defaultdict(None, {'JEF': 0.049000000000000044, 'FULL': 1.0}),
             10: defaultdict(None, {'JEF': 0.18200000000000005, 'FULL': 1.0}),
             20: defaultdict(None, {'JEF': 0.5920000000000001, 'FULL': 1.0}),
             30: defaultdict(None, {'JEF': 0.909, 'FULL': 1.0}),
             40: defaultdict(None, {'JEF': 0.989, 'FULL': 1.0})})


In [31]:
0.182+0.909+0.989+0.692

2.7720000000000002